<a href="https://colab.research.google.com/github/Shreya-Junagade/Student-performance-analyser/blob/main/Student_performance_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import json

class StudentAnalyzer:
    def __init__(self):
        #self.quiz_endpoint = "https://jsonkeeper.com/b/LLQT"
        #self.submission_endpoint = "https://api.jsonserve.com/rJvd7g"
        #self.history_endpoint = "https://api.jsonserve.com/XgAgFJ"

        self.current_quiz_data = None
        self.historical_data = None
        self.topic_performance = {}
        self.difficulty_performance = {}

    def load_data(self, current_quiz: Dict, historical_quizzes: List[Dict]):
        """
        Load and structure quiz data for analysis
        """

        self.current_quiz_data = pd.DataFrame(current_quiz)

        historical_df = []
        for quiz in historical_quizzes:
            df = pd.DataFrame(quiz)
            historical_df.append(df)
        self.historical_data = pd.concat(historical_df, ignore_index=True)

    def analyze_performance(self) -> Dict:
        """
        Analyze performance across different dimensions
        """
        # Topic-wise analysis
        self.topic_performance = self._analyze_topic_performance()

        # Difficulty-wise analysis
        self.difficulty_performance = self._analyze_difficulty_performance()

        # Time-based improvement analysis
        improvement_trends = self._analyze_improvement_trends()

        return {
            'topic_performance': self.topic_performance,
            'difficulty_performance': self.difficulty_performance,
            'improvement_trends': improvement_trends
        }

    def _analyze_topic_performance(self) -> Dict:
        """
        Analyze performance by topic
        """
        topic_stats = {}
        for topic in self.current_quiz_data['topic'].unique():
            topic_questions = self.current_quiz_data[self.current_quiz_data['topic'] == topic]
            topic_stats[topic] = {
                'accuracy': (topic_questions['correct'].sum() / len(topic_questions)) * 100,
                'avg_time': topic_questions['time_taken'].mean(),
                'question_count': len(topic_questions)
            }
        return topic_stats

    def _analyze_difficulty_performance(self) -> Dict:
        """
        Analyze performance by difficulty level
        """
        difficulty_stats = {}
        for level in self.current_quiz_data['difficulty'].unique():
            level_questions = self.current_quiz_data[self.current_quiz_data['difficulty'] == level]
            difficulty_stats[level] = {
                'accuracy': (level_questions['correct'].sum() / len(level_questions)) * 100,
                'avg_time': level_questions['time_taken'].mean(),
                'question_count': len(level_questions)
            }
        return difficulty_stats

    def _analyze_improvement_trends(self) -> Dict:
        """
        Analyze improvement trends over the last 5 quizzes
        """
        quiz_scores = self.historical_data.groupby('quiz_id')['correct'].mean().values
        return {
            'trend': np.polyfit(range(len(quiz_scores)), quiz_scores, 1)[0],
            'scores': quiz_scores.tolist()
        }

    def generate_recommendations(self) -> Dict:
        """
        Generate personalized recommendations based on analysis
        """
        recommendations = {
            'focus_topics': [],
            'practice_suggestions': [],
            'study_strategy': []
        }

        # Identify weak topics (accuracy < 60%)
        weak_topics = [
            topic for topic, stats in self.topic_performance.items()
            if stats['accuracy'] < 60
        ]
        recommendations['focus_topics'] = weak_topics

        # Generate practice suggestions
        for difficulty, stats in self.difficulty_performance.items():
            if stats['accuracy'] < 70:
                recommendations['practice_suggestions'].append(
                    f"Practice more {difficulty} level questions"
                )

        # Study strategy based on performance patterns
        if len(weak_topics) > 2:
            recommendations['study_strategy'].append(
                "Focus on fundamentals in weak topics before advancing"
            )

        return recommendations

    def define_student_persona(self) -> Dict:
        """
        Define student persona based on performance patterns
        """
        persona = {
            'learning_style': self._determine_learning_style(),
            'strengths': self._identify_strengths(),
            'areas_for_improvement': self._identify_improvements(),
            'performance_label': self._assign_performance_label()
        }
        return persona

    def predict_neet_rank(self, previous_year_cutoffs: Dict) -> Dict:
        """
        Predict potential NEET rank range based on performance
        """
        # Calculate overall performance metrics
        avg_accuracy = np.mean([stats['accuracy'] for stats in self.topic_performance.values()])
        consistency = np.std([stats['accuracy'] for stats in self.topic_performance.values()])

        # Simple probabilistic model
        base_percentile = avg_accuracy * (1 - consistency/100)

        # Calculate rank range based on previous year cutoffs
        rank_range = self._calculate_rank_range(base_percentile, previous_year_cutoffs)

        return {
            'predicted_percentile': base_percentile,
            'rank_range': rank_range,
            'confidence_level': 'medium'  # Could be refined based on more factors
        }

    def visualize_performance(self):
        """
        Create visualizations for performance analysis
        """
        # Create figure with subplots
        fig = plt.figure(figsize=(15, 10))

        # Topic Performance
        plt.subplot(2, 2, 1)
        topics = list(self.topic_performance.keys())
        accuracies = [stats['accuracy'] for stats in self.topic_performance.values()]
        plt.bar(topics, accuracies)
        plt.title('Topic-wise Performance')
        plt.xticks(rotation=45)
        plt.ylabel('Accuracy (%)')

        # Difficulty Level Performance
        plt.subplot(2, 2, 2)
        difficulties = list(self.difficulty_performance.keys())
        diff_accuracies = [stats['accuracy'] for stats in self.difficulty_performance.values()]
        plt.bar(difficulties, diff_accuracies)
        plt.title('Performance by Difficulty Level')
        plt.ylabel('Accuracy (%)')

        # Improvement Trend
        plt.subplot(2, 2, 3)
        trend_data = self._analyze_improvement_trends()
        plt.plot(range(len(trend_data['scores'])), trend_data['scores'], marker='o')
        plt.title('Performance Trend (Last 5 Quizzes)')
        plt.xlabel('Quiz Number')
        plt.ylabel('Score')

        # Time vs Accuracy Scatter
        plt.subplot(2, 2, 4)
        plt.scatter(
            self.current_quiz_data['time_taken'],
            self.current_quiz_data['correct']
        )
        plt.title('Time vs Accuracy Correlation')
        plt.xlabel('Time Taken (seconds)')
        plt.ylabel('Correct (1/0)')

        plt.tight_layout()
        return fig

    def _determine_learning_style(self) -> str:
        # Analyze time taken vs accuracy patterns
        time_accuracy_corr = np.corrcoef(
            self.current_quiz_data['time_taken'],
            self.current_quiz_data['correct']
        )[0, 1]

        if time_accuracy_corr > 0.5:
            return "Methodical Learner"
        elif time_accuracy_corr < -0.5:
            return "Intuitive Learner"
        return "Balanced Learner"

    def _identify_strengths(self) -> List[str]:
        return [
            topic for topic, stats in self.topic_performance.items()
            if stats['accuracy'] >= 80
        ]

    def _identify_improvements(self) -> List[str]:
        return [
            topic for topic, stats in self.topic_performance.items()
            if stats['accuracy'] < 60
        ]

    def _assign_performance_label(self) -> str:
        avg_accuracy = np.mean([stats['accuracy'] for stats in self.topic_performance.values()])
        if avg_accuracy >= 80:
            return "Advanced Performer"
        elif avg_accuracy >= 60:
            return "Steady Learner"
        return "Building Foundations"

    def _calculate_rank_range(self, percentile: float, previous_cutoffs: Dict) -> Tuple[int, int]:
        # Simple rank range calculation based on percentile and previous year cutoffs
        base_rank = int(percentile * 1000)  # Assuming total candidates = 100,000
        return (base_rank - 1000, base_rank + 1000)  # ±1000 rank range



def main():

    current_quiz = {
        'question_id': [1, 2, 3, 4, 5],
        'topic': ['Physics', 'Chemistry', 'Biology', 'Physics', 'Chemistry'],
        'difficulty': ['easy', 'medium', 'hard', 'medium', 'hard'],
        'correct': [1, 1, 0, 0, 1],
        'time_taken': [60, 90, 120, 80, 100]
    }

    historical_quizzes = [
        {
            'quiz_id': [1] * 5,
            'correct': [1, 1, 0, 1, 1]
        },
        {
            'quiz_id': [2] * 5,
            'correct': [1, 1, 1, 0, 1]
        }
    ]

    previous_year_cutoffs = {
        '90_percentile': 50000,
        '80_percentile': 100000,
        '70_percentile': 150000
    }

    analyzer = StudentAnalyzer()
    analyzer.load_data(current_quiz, historical_quizzes)

    # Generate analysis
    performance_analysis = analyzer.analyze_performance()
    recommendations = analyzer.generate_recommendations()
    persona = analyzer.define_student_persona()
    rank_prediction = analyzer.predict_neet_rank(previous_year_cutoffs)

    # Create visualizations
    analyzer.visualize_performance()
    plt.show()

    # Print results
    print("Performance Analysis:", json.dumps(performance_analysis, indent=2))
    print("\nRecommendations:", json.dumps(recommendations, indent=2))
    print("\nStudent Persona:", json.dumps(persona, indent=2))
    print("\nRank Prediction:", json.dumps(rank_prediction, indent=2))




    current_data = {
        'question_id': range(1, 11),
        'topic': ['Physics']*3 + ['Chemistry']*4 + ['Biology']*3,
        'concept': ['Mechanics', 'Thermodynamics', 'Waves', 'Organic', 'Inorganic',
                   'Physical', 'Organic', 'Genetics', 'Ecology', 'Genetics'],
        'difficulty': ['easy', 'medium', 'hard'] * 3 + ['medium'],
        'correct': [1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
        'time_taken': [45, 60, 90, 75, 60, 45, 90, 60, 75, 45]
    }

    historical_data = [
        {
            'quiz_id': [1] * 5,
            'correct': [1, 1, 0, 1, 1],
            'topic': ['Physics', 'Chemistry', 'Biology', 'Physics', 'Chemistry']
        }
    ]




if __name__ == "__main__":
    main()


In [ ]:
class PerformanceGapAnalyzer:
    def __init__(self, current_data: Dict, historical_data: List[Dict]):
        """
        Initialize with current quiz data and historical quiz data.
        """
        self.current_data = pd.DataFrame(current_data)
        self.historical_data = pd.concat([pd.DataFrame(quiz) for quiz in historical_data], ignore_index=True)

        self.topic_gaps = {}
        self.concept_mastery = {}
        self.difficulty_gaps = {}

    def analyze_weak_areas(self) -> Dict:
        """
        Analyze weak areas across different dimensions like topics, concepts, and difficulty levels.
        """
        self.topic_gaps = self._analyze_topic_gaps()
        self.concept_mastery = self._analyze_concept_mastery_gaps()
        self.difficulty_gaps = self._analyze_difficulty_gaps()

        return {
            'topic_gaps': self.topic_gaps,
            'concept_mastery': self.concept_mastery,
            'difficulty_gaps': self.difficulty_gaps
        }

    def _analyze_topic_gaps(self) -> Dict:
        """
        Analyze performance by topic to identify weak areas.
        """
        topic_stats = {}
        for topic in self.current_data['topic'].unique():
            topic_questions = self.current_data[self.current_data['topic'] == topic]
            accuracy = (topic_questions['correct'].sum() / len(topic_questions)) * 100
            topic_stats[topic] = {
                'accuracy': accuracy,
                'is_weak': accuracy < 60  # Consider topics with < 60% accuracy as weak
            }
        return topic_stats

    def _analyze_concept_mastery_gaps(self) -> Dict:
        """
        Analyze performance by concept to identify mastery gaps.
        """
        concept_stats = {}
        for concept in self.current_data['concept'].unique():
            concept_questions = self.current_data[self.current_data['concept'] == concept]
            accuracy = (concept_questions['correct'].sum() / len(concept_questions)) * 100
            concept_stats[concept] = {
                'mastery_level': accuracy,
                'needs_attention': accuracy < 60  # Consider concepts with < 60% mastery as needing attention
            }
        return concept_stats

    def _analyze_difficulty_gaps(self) -> Dict:
        """
        Analyze performance by difficulty level to identify areas that need focus.
        """
        difficulty_stats = {}
        for level in self.current_data['difficulty'].unique():
            level_questions = self.current_data[self.current_data['difficulty'] == level]
            accuracy = (level_questions['correct'].sum() / len(level_questions)) * 100
            difficulty_stats[level] = {
                'accuracy': accuracy,
                'needs_focus': accuracy < 60  # Consider difficulty levels with < 60% accuracy as needing focus
            }
        return difficulty_stats

    def visualize_gaps(self):
        """
        Create visualizations to highlight weak areas.
        """
        fig = plt.figure(figsize=(15, 12))

        # Topic Gaps Visualization
        plt.subplot(2, 2, 1)
        topics = list(self.topic_gaps.keys())
        topic_accuracies = [data['accuracy'] for data in self.topic_gaps.values()]
        plt.bar(topics, topic_accuracies, color=['red' if data['is_weak'] else 'green' for data in self.topic_gaps.values()])
        plt.title('Topic-wise Performance Gaps')
        plt.xticks(rotation=45)
        plt.ylabel('Accuracy (%)')

        # Concept Mastery Visualization
        plt.subplot(2, 2, 2)
        concepts = list(self.concept_mastery.keys())
        concept_accuracies = [data['mastery_level'] for data in self.concept_mastery.values()]
        plt.bar(concepts, concept_accuracies, color=['red' if data['needs_attention'] else 'green' for data in self.concept_mastery.values()])
        plt.title('Concept Mastery Gaps')
        plt.xticks(rotation=45)
        plt.ylabel('Mastery Level (%)')

        # Difficulty Level Gaps Visualization
        plt.subplot(2, 2, 3)
        difficulty_levels = list(self.difficulty_gaps.keys())
        difficulty_accuracies = [data['accuracy'] for data in self.difficulty_gaps.values()]
        plt.bar(difficulty_levels, difficulty_accuracies, color=['red' if data['needs_focus'] else 'green' for data in self.difficulty_gaps.values()])
        plt.title('Difficulty Level Gaps')
        plt.ylabel('Accuracy (%)')

        plt.tight_layout()
        return fig


In [ ]:
current_data = {
    'question_id': range(1, 11),
    'topic': ['Physics']*3 + ['Chemistry']*4 + ['Biology']*3,
    'concept': ['Mechanics', 'Thermodynamics', 'Waves', 'Organic', 'Inorganic',
               'Physical', 'Organic', 'Genetics', 'Ecology', 'Genetics'],
    'difficulty': ['easy', 'medium', 'hard'] * 3 + ['medium'],
    'correct': [1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
    'time_taken': [45, 60, 90, 75, 60, 45, 90, 60, 75, 45]
}

historical_data = [
    {
        'quiz_id': [1] * 5,
        'correct': [1, 1, 0, 1, 1],
        'topic': ['Physics', 'Chemistry', 'Biology', 'Physics', 'Chemistry']
    }
]

analyzer = PerformanceGapAnalyzer(current_data, historical_data)

gap_analysis = analyzer.analyze_weak_areas()

print("\nTopic Gaps:")
for topic, data in gap_analysis['topic_gaps'].items():
    if data['is_weak']:
        print(f"- {topic}: {data['accuracy']:.1f}% accuracy (Weak Area)")

print("\nConcept Mastery Gaps:")
for concept, data in gap_analysis['concept_mastery'].items():
    if data['needs_attention']:
        print(f"- {concept}: {data['mastery_level']:.1f}% mastery (Needs Attention)")

print("\nDifficulty Level Gaps:")
for level, data in gap_analysis['difficulty_gaps'].items():
    if data['needs_focus']:
        print(f"- {level}: {data['accuracy']:.1f}% accuracy (Needs Focus)")

# Visualize the gaps
analyzer.visualize_gaps()
plt.show()
